In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Import from BigQuery into Vector Search

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/embeddings/bigquery-import.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fembeddings%2Fbigquery-import.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/bigquery-import.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/embeddings/bigquery-import.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/bigquery-import.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/bigquery-import.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/bigquery-import.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/bigquery-import.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/bigquery-import.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>            

| Authors |
| --- |
| [Eric Gribkoff](https://github.com/ericgribkoff) |

### Objectives

In this notebook, you will learn how to import vector embedding data from a BigQuery data source into a Vector Search index.

## Getting Started

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).

In [ ]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com) and [enable the BigQuery API](https://console.cloud.google.com/flows/enableapi?apiid=bigquery.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
# Use the environment variable if the user doesn't provide Project ID.
import os

PROJECT_ID = "your-project-id"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

# Initialize the aiplatform package
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)


## Generate sample BigQuery data



In [ ]:
%%bigquery --project $PROJECT_ID
CREATE SCHEMA import_example_dataset; -- OPTIONS (location=$LOCATION);
CREATE TABLE import_example_dataset.test_table (
  id INTEGER,
  embedding ARRAY <FLOAT64>,
  allow_column STRING,
  deny_column STRING,
  int_column INTEGER,
  float_column FLOAT64,
  metadata_column STRING
);
INSERT INTO import_example_dataset.test_table(id, embedding, allow_column, deny_column, int_column, float_column, metadata_column) VALUES
(1, [0.1, 0.1, 0.1], "allow1", "deny1", 1, 0.1, "metadata1"),
(2, [0.2, 0.2, 0.2], "allow2", "deny2", 2, 0.2, "metadata2");
SELECT * FROM import_example_dataset.test_table;

## Create a Vector Search Index

This may take a few moments to finish the index creation.

In [ ]:
my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name='import_test_index_name',
    dimensions=3,
    approximate_neighbors_count=10,
    index_update_method="BATCH_UPDATE",
)

## Import from BigQuery into Vector Search

This returns a Long-Running Operation (LRO), which allows you to track the progress of the import operation.

In [ ]:
import requests

# Get token to use for REST request
gcloud_token = !gcloud auth print-access-token

url = f"https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{my_index.resource_name}:import"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {gcloud_token[0]}"
}
request = {
  "is_complete_overwrite": True,
  "config": {
    "big_query_source_config": {
      "table_path": f"bq://{PROJECT_ID}.import_example_dataset.test_table",
      "datapoint_field_mapping": {
        "id_column": "id",
        "embedding_column": "embedding",
        "restricts": [
          {"namespace": "restrict", "allow_column": ["allow_column"], "deny_column": ["deny_column"]},
        ],
        "numeric_restricts": [
          {"namespace": "int_restrict", "value_column": "int_column", "value_type": "INT"},
          {"namespace": "float_restrict", "value_column": "float_column", "value_type": "FLOAT"}
        ]
        # The import may include metadata if your project has been allow-listed
        # for the VS metadata preview.
        #"embedding_metadata": "metadata_column"
      }
    }
  }
}

try:
    # Make the POST request
    response = requests.post(url, headers=headers, json=request)

    # Check the response status code
    if response.status_code == 200:
        print("Import request successful!")
        print(response.json())
    else:
        print(f"Import request failed with status code: {response.status_code}")
        print(response.text)

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")


The code below will query the API for the status of the import LRO.



In [ ]:
operation = response.json()['name']
response = requests.get(f"https://{LOCATION}-aiplatform.googleapis.com/v1beta1/{operation}", headers=headers)

if 'done' in response.json():
  if 'error' in response.json():
    print("Import failed")
    print(response.json()['error'])
  else:
    print("Import succeeded!")
    print(response.json())
else:
    print("Import still in progress")
print(response.json())
